<h3></h3>

In [ ]:
import pandas as pd
import os
import seaborn as sns

# ? Se usa para generar gráficos
import matplotlib.pyplot as plt

# ? Se usa para determina la fecha de ejecución del proceso dentro del export de los resultados estadísticos
from datetime import datetime

# ? Conexión a BD Postgres. Se usa dado que pandas no acepta sino este paquete para la conexión.
from sqlalchemy import create_engine, text

from datetime import datetime

from arcgis.features import GeoAccessor, GeoSeriesAccessor
import arcpy

from pathlib import Path

# ** Librerías propias
import funcion_dataframe_terrenos
import funcion_SQL_a_DataFrame
import funcion_parametrizacionPredioMatriz_X_NPN

In [ ]:
DIRECTORIO_REPORTE = r"C:\docsProyectos\5.RAISS\2024.0.RAISS_Lote_4\6.Hitos\E2_Informes_Id_FisicoJuirica\2_2_0_Identificacion_Predial_Total_Ha_Actualizadas\zReportes"
NOMBRE_REPORTE_HASHEET = 'Reporte_Ha_FuenteSheet.xlsx'

RUTA_BD_LOCAL = r"C:\docsProyectos\5.RAISS\2024.0.RAISS_Lote_4\6.Hitos\E2_Informes_Id_FisicoJuirica\2_2_0_Identificacion_Predial_Total_Ha_Actualizadas\Base_Datos\_2_2_0.gdb"
RUTA_BD_CONSOLIDADA = r"C:\docsProyectos\5.RAISS\2024.0.RAISS_Lote_4\6.Hitos\E1_Alistamiento_Diagnostico\3_Disposicion\1.BD_Consolidada\BD_Consolidada_Lote4.gdb"
NOMBRE_CAPA_HAXESTADO = 'TERRENOS_RECONOCIMIENTO_TRAMITES_ATENCION_SHEET'

In [ ]:
sheet_name = 'Rendimientos_Reconocimiento_MariaLaBaja'
sheet_id = '1-6JLJwBQk5O57VU11kD02mmsowbyL2rJ9HzsN2XfNxs'

url = f"https://docs.google.com/spreadsheets/d/{sheet_id}/gviz/tq?tqx=out:csv&sheet={sheet_name}"

df_reporte_reconocimiento = pd.read_csv(url)
columnas_seleccion = ['MUNICIPIO', 
    'ZONA', 
    'CODIGO',
    'Area terreno_Ha',
    'CODIGO_MUN', 
    'COORDINADOR',
    'RECONOCEDOR',
    'PROGRAMACION',
    'Visitado',
    'Nº Radicado BCGS']

df_reporte_reconocimiento = df_reporte_reconocimiento[columnas_seleccion]

df_reporte_reconocimiento = df_reporte_reconocimiento.copy()

total_registros = df_reporte_reconocimiento.shape[0]
print(f"El total de registros analizados es de {total_registros}")

In [ ]:
df_reporte_reconocimiento = df_reporte_reconocimiento[(df_reporte_reconocimiento['Visitado'] == 'SI')]

<h3>Ejecución de Funciones</h3>

In [ ]:

df_terrenos = funcion_dataframe_terrenos.dataframe_terrenos()

<h3>Espacialización de Rendimientos Reconocimiento</h3>

In [ ]:

# ** Left para verificar posteriormente cuales terrenos son posibles informalidades
df_terrenos_reporteReconocimiento = pd.merge(left=df_reporte_reconocimiento, right=df_terrenos, on="CODIGO", how="left")
print(f"{df_terrenos_reporteReconocimiento.shape[0]}")

<h3>Análisis de Terrenos Visitados sin Geometrías</h3>

In [ ]:
df_terrenos_reporteReconocimiento_sinGeo =  df_terrenos_reporteReconocimiento[df_terrenos_reporteReconocimiento['SHAPE'].isnull()]
terrenos_posiblesInformalidades = df_terrenos_reporteReconocimiento_sinGeo.shape[0]
print(f"Predios sin geometrías, posibles informalidades: {df_terrenos_reporteReconocimiento_sinGeo.shape[0]}")

<h4>Búsqueda del Predio Matriz a partir del NPN</h4>

In [ ]:

# ** Aplica la función a cada elemento de la columna npn y asigna el resultado a npn_parametrizado

df_terrenos_reporteReconocimiento_sinGeo = df_terrenos_reporteReconocimiento_sinGeo.copy()
df_terrenos_reporteReconocimiento_sinGeo['npn_matriz'] = df_terrenos_reporteReconocimiento_sinGeo['CODIGO'].apply(funcion_parametrizacionPredioMatriz_X_NPN.eliminacion_mejora)

df_terrenos_reporteReconocimiento_sinGeo_sinMatriz = df_terrenos_reporteReconocimiento_sinGeo[df_terrenos_reporteReconocimiento_sinGeo['npn_matriz'].isnull()]
df_terrenos_reporteReconocimiento_sinGeo_conMatriz = df_terrenos_reporteReconocimiento_sinGeo[df_terrenos_reporteReconocimiento_sinGeo['npn_matriz'].notnull()]

print(f"Terrenos por verificar desde Reconocimiento: {df_terrenos_reporteReconocimiento_sinGeo_sinMatriz}")

<h4>Espacializacion de los posibles Matrices</h4>

In [ ]:
reporteReconocimiento_sinGeo_conMatriz_Unicos = pd.DataFrame(df_terrenos_reporteReconocimiento_sinGeo_conMatriz['npn_matriz'].value_counts()).rename(columns={'count':'informalidades_visitadas'}).reset_index()
predios_matrices = reporteReconocimiento_sinGeo_conMatriz_Unicos.shape[0]
print(f"Total de posibles Matrices: {reporteReconocimiento_sinGeo_conMatriz_Unicos.shape[0]}")

reporteReconocimiento_sinGeo_conMatriz_Unicos_Geo = pd.merge(left=reporteReconocimiento_sinGeo_conMatriz_Unicos, right=df_terrenos, left_on="npn_matriz", right_on="CODIGO", how="inner")
print(f"Matrices espacializados: {reporteReconocimiento_sinGeo_conMatriz_Unicos_Geo}")

<h4>Calculo de Ha Posibles Matrices (del tratamiento de informalidades)</h4>

In [ ]:
total_ha_matrices = reporteReconocimiento_sinGeo_conMatriz_Unicos_Geo['Area_ha_cmt12'].sum().round(2)
print(f"Total Ha Predios Matrices: {total_ha_matrices} Ha")

<h3>Análisis de Terrenos Visitados con Geometrías</h3>

In [ ]:
df_terrenos_reporteReconocimiento_conGeo =  df_terrenos_reporteReconocimiento[df_terrenos_reporteReconocimiento['SHAPE'].notnull()]
terrenos_formales = df_terrenos_reporteReconocimiento_conGeo.shape[0]
print(f"Predios con geometrías: {df_terrenos_reporteReconocimiento_conGeo.shape[0]}")

In [ ]:
total_ha_conGeo = df_terrenos_reporteReconocimiento_conGeo['Area_ha_cmt12'].sum().round(2)
print(f"Total Ha Predios Matrices: {total_ha_conGeo} Ha")

In [ ]:
municipio = 'Maria La Baja'

df_resumen_marialabaja_ha = pd.DataFrame([[municipio, terrenos_posiblesInformalidades, predios_matrices, total_ha_matrices, terrenos_formales, total_ha_conGeo]], columns=['municipio', 'posibles_informalidades', 'terrenos_matrices', 'total_ha_posiblesinformalidades','predios_formales', 'total_ha_prediosformales'])
df_resumen_marialabaja_ha['total_ha'] = df_resumen_marialabaja_ha['total_ha_posiblesinformalidades'] + df_resumen_marialabaja_ha['total_ha_prediosformales']

In [ ]:

reporteReconocimiento_sinGeo_conMatriz_Unicos_Geo = reporteReconocimiento_sinGeo_conMatriz_Unicos_Geo.copy()
df_terrenos_reporteReconocimiento_conGeo = df_terrenos_reporteReconocimiento_conGeo.copy()

# TODO: Matrices
# ** Exportación a BD Consolidada
NOMBRE_CAPA_SALIDA = f"Analitica_UT_Consolidada\\{'TERRENOS_RECONOCIMIENTO_TRAMITES_ATENCION_SHEET_MATRICES'.upper()}"
ruta_salida_terrenos_reconocimiento_matrices = os.path.join(RUTA_BD_CONSOLIDADA, NOMBRE_CAPA_SALIDA)

reporteReconocimiento_sinGeo_conMatriz_Unicos_Geo.spatial.to_featureclass(location=ruta_salida_terrenos_reconocimiento_matrices)
print(f"Se exporta la capa {NOMBRE_CAPA_SALIDA} a BD Consolidada")

# TODO: Formales
# ** Exportación a BD Consolidada
NOMBRE_CAPA_SALIDA = f"Analitica_UT_Consolidada\\{'TERRENOS_RECONOCIMIENTO_TRAMITES_ATENCION_SHEET_FORMALES'.upper()}"
ruta_salida_terrenos_reconocimiento_formales = os.path.join(RUTA_BD_CONSOLIDADA, NOMBRE_CAPA_SALIDA)

df_terrenos_reporteReconocimiento_conGeo.spatial.to_featureclass(location=ruta_salida_terrenos_reconocimiento_formales)
print(f"Se exporta la capa {NOMBRE_CAPA_SALIDA} a BD Consolidada")

df_resumen_marialabaja_ha.to_excel(os.path.join(DIRECTORIO_REPORTE, NOMBRE_REPORTE_HASHEET))